In [1]:
import sys
import json
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer
import getpass as gt
import pandas as pd
import numpy as np

from snowflake_conn import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
conn = get_connection()

## Figure out why only outpatient claims are populating in cost, even though there are inpatient/snf claims in the Tuva medical claims model

#### Start with raw Tuva medical claim
`dbt_packages/the_tuva_project/models/claims_preprocessing/service_category/staging/service_category__stg_medical_claim.sql`

In [10]:
sql='''
select
APR_DRG_CODE,
BILL_TYPE_CODE,
CLAIM_ID,
CLAIM_LINE_NUMBER,
CLAIM_TYPE,
HCPCS_CODE,
MS_DRG_CODE,
PLACE_OF_SERVICE_CODE,
REVENUE_CENTER_CODE
from SANDBOX_FFS._tuva_claims.medical_claim
limit 1000000
'''

raw_claim = read_sql(sql)
print(raw_claim.shape)
raw_claim.head(1)

(1000000, 9)


,apr_drg_code,bill_type_code,claim_id,claim_line_number,claim_type,hcpcs_code,ms_drg_code,place_of_service_code,revenue_center_code
0,None,None,-10000930168411201871,1,professional,None,None,11,None


#### Go through each type of service grouper
`dbt_packages/the_tuva_project/models/claims_preprocessing/service_category/intermediate/*.sql`
- *Note: the source of truth from these apparently comes from `SANDBOX_FFS._tuva_claims.medical_claim` -- I don't know how this differs from `SANDBOX_FFS.tuva_claims.medical_claim`

Data:
- acute inpatient professional -- (294, 3)
    - place_of_service_code = '21'
- dme professional -- (6303, 3)
    - hcpcs_code between 'E0100' and 'E8002'
- emergency department professional -- (282, 3)
    - place_of_service_code = '23'
- home health institutional -- (493, 2)
    - left(bill_type_code,2) in ('31','32','33')
- home health professional -- (4225, 3)
    - join on dme professional (claim number, claim line number)
    - medical claim place_of_service_code in ('12')
    - (dme claim_id is null and dme claim_line_number is null)
- hospice institutional -- (1086, 2)
    - left(bill_type_code,2) in ('81','82')
- hospice professional -- (2631, 3)
    - place_of_service_code in ('34')
- office visit professional -- (951163, 3)
    - place_of_service_code in ('11','02')
- outpatient hospital or clinic institutional -- (402653, 3)
    - join on emergency dept insitutional --> no data
    - join on urgent care institutional --> no data
    - left(a.bill_type_code,2) in ('13','71','73') --> populates
    - *I don't see what the point of joining on two null tables is, even if we did have did have data there*
- skilled nursing insitutitional -- (1632, 2)
    - left(bill_type_code,2) in ('21','22')
- skilled nursing professional -- (5174, 3)
    - join on dme professional (on claim id and claim line number)
    - place_of_service_code in ('31','32') -- not sure which one
    - (dme claim_id is null and dme claim_line_number is null)
- urgent care professional -- (175672, 3)
    - place_of_service_code in ('20')

No rows:
- acute impatient institutional
    - room and board requirement --> populates ('0100','0101','0110','0111','0112','0113','0114','0116','0117','0118','0119','0120','0121','0122','0123','0124','0126','0127','0128','0129','0130','0131','0132','0133','0134','0136','0137','0138','0139','0140','0141','0142','0143','0144','0146','0147','0148','0149','0150','0151','0152','0153','0154','0156','0157','0158','0159','0160','0164','0167','0169','0170','0171','0172','0173','0174','0179','0190','0191','0192','0193','0194','0199','0200','0201','0202','0203','0204','0206','0207','0208','0209','0210','0211','0212','0213','0214','0219','1000','1001','1002')
    - drg requirement --> does not populate (join medical claim diagnosis with `SANDBOX_FFS.terminology.ms_drg` and `SANDBOX_FFS.terminology.apr_drg`)
    - bill type requirement --> populates left(bill_type_code,2) in ('11','12') 
- ambulence professional
    - (hcpcs_code between 'A0425' and 'A0436' or place_of_service_code in ('41','42'))
- ambualatory surgery professional
    - join on dme professional (on claim id and claim line number) --> this populates
    - medical claim place_of_service_code in ('24') --> this doesn't populate
    - (dme prof claim_id is null and dme prof claim_line_number is null) --> this populates
- dialysis institutional
    - left(bill_type_code,2) in ('72')
- dialysis professional
    - place_of_service_code in ('65')
- emergency department institutional
    - revenue_center_code in ('0450','0451','0452','0459','0981')
    - left(bill_type_code,2) in ('13','71','73') 
    - *Note: 0456, urgent care, is included in most published definitions*
- inpatient psychiatric professional
    - place_of_service_code in ('51','55','56')
- inpatient rehab professional
    - place_of_service_code in ('61')
- lab institutional
    - left(bill_type_code,2) in ('14')
- lab professional
    - place_of_service_code = '81'
- outpatient hospital or clinic professional
    - place_of_service_code in ('15','17','19','22','49','50','60','71','72')
- outpatient psychiatric institutional
    - left(bill_type_code,2) in ('52')
- outpatient psychiatric professional
    - place_of_service_code in ('52','53','57','58')
- outpatient rehab professional
    - place_of_service_code in ('62')
- urgent care insitutional
    - revenue_center_code = '0456'
    - left(bill_type_code,2) in ('13','71','73')

Professional combined -- (1164504, 4)
- YES: 
    - acute inpatient professional -- 294
    - dme professional -- 6303
    - emergency dept professional -- 282
    - home health professional -- 4225
    - hospice professional -- 2631
    - outpatient hospital or clinic -- 18760
    - office visit professional -- 951163
    - skilled nursing professional -- 5174
    - urgent care professional -- 175672
- No:  
    - ambulence professional 
    - ambulatory surgery professional
    - dialysis professional
    - inpatient psychiatric professional
    - inpatient rehab professional
    - lab professional
    - outpatient hospital or clinic professional
    - outpatient psychiatric professional
    - outpatient rehab professional

Insitutional combined -- (405864, 3)
- YES:
    - home health institutional -- 493
    - hospice institutional -- 1086
    - outpatient hospital or clinic -- 402653
    - skilled nursing institutional -- 1632
- No:
    - acute impatient institutional
    - dialysis institutional
    - emergency department institutioanl
    - lab institutional
    - outpatient hospital or clinic institutional
    - outpatient psychiatric institutional
    - urgent care insitutional

In [12]:
loc = 'SANDBOX_FFS._tuva_claims.medical_claim'

In [126]:
# THIS IS STORED IN A VERY WEIRD LOCATION

sql=f'''
select * from SANDBOX_FFS.None.service_category__service_category_grouper
'''
tmp = read_sql(sql)
print(tmp.shape)
tmp.head(1)

(1822814, 5)


,claim_id,claim_line_number,claim_type,service_category_1,service_category_2
0,-10000930682600202071,10,professional,Office Visit,Office Visit


In [127]:
tmp.groupby('claim_type')['service_category_2'].value_counts()

claim_type     service_category_2           
institutional  Outpatient Hospital or Clinic    575092
               Other                             58066
               Skilled Nursing                   12548
               Hospice                           12107
               Home Health                        6215
professional   Office Visit                     948381
               Urgent Care                      175333
               Outpatient Hospital or Clinic     17085
               Durable Medical Equipment          6303
               Skilled Nursing                    5174
               Home Health                        4225
               Hospice                            2041
               Emergency Department                201
               Other                                43
Name: count, dtype: int64

In [125]:
both.groupby('type')['service_category_2'].value_counts()

type           service_category_2           
institutional  Outpatient Hospital or Clinic    402653
               Skilled Nursing                    1632
               Hospice                            1086
               Home Health                         493
professional   Office Visit                     951163
               Urgent Care                      175672
               Outpatient Hospital or Clinic     18760
               Durable Medical Equipment          6303
               Skilled Nursing                    5174
               Home Health                        4225
               Hospice                            2631
               Acute Inpatient                     294
               Emergency Department                282
Name: count, dtype: int64

In [115]:
sql=f'''
select * from SANDBOX_FFS.claims_preprocessing._int_combined_professional
'''
prof = read_sql(sql)
print(prof.shape)
prof.groupby('service_category_2').size()

(1164504, 4)


service_category_2
Acute Inpatient                     294
Durable Medical Equipment          6303
Emergency Department                282
Home Health                        4225
Hospice                            2631
Office Visit                     951163
Outpatient Hospital or Clinic     18760
Skilled Nursing                    5174
Urgent Care                      175672
dtype: int64

In [117]:
sql=f'''
select * from SANDBOX_FFS.claims_preprocessing._int_combined_institutional
'''
inst = read_sql(sql)
print(inst.shape)
inst.groupby('service_category_2').size()

(405864, 3)


service_category_2
Home Health                         493
Hospice                            1086
Outpatient Hospital or Clinic    402653
Skilled Nursing                    1632
dtype: int64

In [ ]:
prof['type'] = 'professional'
inst['type'] = 'institutional'
both = pd.concat([prof,inst],ignore_index=True)
both.groupby('type')['service_category_2'].value_counts()